# CLASSIFICATION OF THE MF AND GRC ACTIVITY 

In [ ]:
## Perceptron classification for the MF and GrC spiking activity 

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import os
import sys
import datetime
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import keras.backend as K


In [ ]:
##################### PARAMETERS #########################

########## 01_anatomical_model_generation ################

seed = 3 # seed for the generation of cells position

########## 02_input_patterns_generation ##################

## Spatial correlation between MFs 
sigma = 5 

## List of MF active fractions that will be used 
f_mf = np.linspace(.05, 0.95, 10)

## Number of patterns to be simulate for each
## MF active fraction 
num_patterns = 640

################ 03_simulations ##########################

## Interval used to count spikes 
## during each pattern in the EDLUT simulation
dt = interval = 0.070 

## List of noises fractions to be tested
noises = [0.2]

## Total synaptic weight that reaches each GrC from MFs
mf_grc_w = 4.00

# Inhibitory weight in GoC to GrC synapsis
goc_grc_w = 0.50 

## List of synaptic weights to simulate from MF to GoC
mf_goc_weights = [0.0, 0.10]

## List of synaptic weights to simulate from GrC to GoC
grc_goc_weights = [0.02]

## Number of presentations/samples for each pattern
## Important parameter when noise != 0.0
n_seeds = 5 

################ 04_perceptron ###########################

## Perceptron seed, used when splitting the dataset
## in training, validation and test
perceptron_seed = seed

## Number of classes for classification
C = 10 

## Backpropagation parameters
gamma = 0.001 # learning rate of the perceptron
batch_size = 32
N_epochs = 500 #Number of epochs


############################################################

In [ ]:
sys.path.insert(1, '../data/spikes')
# Function to get MF spikes samples 
def get_samples_mf(fraction, noise, k):
    
    x_mf = np.loadtxt('../data/spikes/seed' + str(seed) 
                        + '/sigma_' + str(sigma) 
                        + '/noise_' + str(noise)
                        + '/mf'
                        + '/f_mf_' + str(fraction) 
                        + '/i' + str(interval) + '_mf_s' + str(k) + '.csv')

    return x_mf

# Function to get GrC spikes samples
def get_samples_grc(fraction, noise, mf_goc_w, grc_goc_w, k):

    x_grc = np.loadtxt('../data/spikes/seed' + str(seed) 
                        + '/sigma_' + str(sigma) 
                        + '/noise_' + str(noise)
                        + '/grc'
                        + '/mf_grc_' + str(mf_grc_w) 
                        + '/mf_goc_w_' + str(mf_goc_w)
                        + '/goc_grc_w_' + str(goc_grc_w)
                        + '/grc_goc_w_' + str(grc_goc_w)
                        +'/f_mf_' + str(fraction) 
                        + '/i' + str(interval) + '_grc_s' + str(k) + '.csv')
    
    return x_grc



In [ ]:
## Dataframe columns names for saving the accuracies 
columns = ["positions_seed", "perceptron_seed", "sigma", "fraction", 
                            "noise", "mf_grc_w", 
                            "mf_goc_w", "grc_goc_w", "goc_grc_w", "interval", "N_epochs",
                            "C", "num_patterns", "n_seeds", "gamma", "batch_size", "type", "filename"]

In [ ]:


## For each noise fraction in noises list
for noise in noises:

    ## For each MF active fraction 
    for elem in f_mf:

        fraction = elem
        
        ## Setting early stopping 
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        ## Initializing seed
        np.random.seed(seed)
        keras.utils.set_random_seed(perceptron_seed)
        
        ########## MF perceptron classification ##############################################
        
        print("")
        print('Sigma: ' + str(sigma))
        print('f_mf: ' + str(fraction))
        print("")
    
        ## Defining optimizer
        opt = keras.optimizers.SGD(learning_rate=gamma)
            
        
        # Defining the single layer of the perceptron 
        x = get_samples_mf(fraction, noise, 1) 
        model = keras.Sequential([
            keras.layers.Dense(C, input_shape=(x.shape[0],),
                            activation='softmax', kernel_initializer='random_uniform')]) 

        # Using accuracy as metric
        model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

        
        ## Creating the datasets by joining the samples for all the presentations (n_seeds)
        
        ## Assigning each pattern to a class
        y = np.zeros((C,num_patterns))
        for k in range(num_patterns):
            y[np.random.choice(C),k] = 1
        y = np.tile(y, reps = n_seeds)

        ## Joining datasets
        x = get_samples_mf(fraction, noise, 0)
        x = x[:,:num_patterns]
        for i in range(n_seeds - 1): 
            x1 = get_samples_mf(fraction, i+1)
            x1 = x1[:,:num_patterns]
            x = np.concatenate((x, x1), axis = 1)
           
        
        ## Splitting datasets in train, val and test (70%, 10% and 20%)
        x_train, x_test, y_train, y_test = train_test_split(x.T, y.T, test_size=0.2, random_state=perceptron_seed, stratify=y.T)
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.125, random_state=perceptron_seed, stratify=y_train)
    
        ## Fitting the model 
        history = model.fit(x_train, y_train, 
                            epochs=N_epochs, 
                            validation_data=(x_val, y_val), 
                            batch_size=batch_size, 
                            callbacks=[callback], verbose = 0)

        ## Retrieving accuracy values
        acc_train_mf = history.history['accuracy']
        acc_val_mf = history.history['val_accuracy']
        acc_test_mf = model.evaluate(x_test, y_test)[-1]

        print('Accuracies:')
        print('Training MF: ' + str(acc_train_mf[-1]))
        print('Validation MF: ' + str(acc_val_mf[-1]))
        print('Test MF: ' + str(acc_test_mf))

        ########## GrC perceptron classification ##############################################
        
        ## For each MF - GoC weight
        for mf_goc_w in mf_goc_weights:
            
            ## For each GrC - GoC weight
            for grc_goc_w in grc_goc_weights: 

                print("")
                print('MF - GOC weight: ' + str(mf_goc_w))
                print("")
                print('GRC - GOC weight: ' + str(grc_goc_w))
                    
                sys.path.insert(1, '../results')

                 
                # Defining optimizer
                opt = keras.optimizers.SGD(learning_rate=gamma)

                # Defining the single layer of the perceptron
                x = get_samples_grc(fraction, noise, mf_goc_w, grc_goc_w, 1)
                model = keras.Sequential([
                    keras.layers.Dense(C, input_shape=(x.shape[0],),
                                    activation='softmax', kernel_initializer='random_uniform')]) 


                # Using accuracy as metric
                model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

                
                ## Creating the datasets by joining the samples for all the presentation (n_seeds)
                
                ## Assigning each pattern to a class
                y = np.zeros((C,num_patterns))
                for k in range(num_patterns):
                    y[np.random.choice(C),k] = 1
                y = np.tile(y, reps = n_seeds)

                ## Joining datasets
                x = get_samples_grc(fraction, noise, mf_goc_w, grc_goc_w, 0)
                x = x[:,:num_patterns]
                for i in range(n_seeds - 1): 
                    x1 = get_samples_grc(fraction, noise, mf_goc_w, grc_goc_w, i+1)
                    x1 = x1[:,:num_patterns]
                    x = np.concatenate((x, x1), axis = 1)

                    
                ## Splitting datasets in train, val and test (70%, 10% and 20%)
                x_train, x_test, y_train, y_test = train_test_split(x.T, y.T, test_size=0.2, random_state=perceptron_seed, stratify=y.T)
                x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.125, random_state=perceptron_seed, stratify=y_train)

                ## Fitting the model 
                history = model.fit(x_train, y_train, 
                                    epochs=N_epochs, 
                                    validation_data=(x_val, y_val), 
                                    batch_size=batch_size, 
                                    callbacks=[callback], verbose = 0)
                               
                ## Retrieving accuracy values
                acc_train_grc = history.history['accuracy']
                acc_val_grc = history.history['val_accuracy']
                acc_test_grc = model.evaluate(x_test, y_test)[-1]


                print('Accuracies:')
                print('Training GRC: ' + str(acc_train_grc[-1]))
                print('Validation GRC: ' + str(acc_val_grc[-1]))
                print('Test GRC: ' + str(acc_test_grc))

                acc_train = [acc_train_mf, acc_train_grc]
                acc_val = [acc_val_mf, acc_val_grc]
                acc_test = [acc_test_mf, acc_test_grc]
               
                
                ## Creating the directory for accuracies
                RESULTS_DIR = '../results/accuracies' + '/seed' + str(seed) 
                os.makedirs(RESULTS_DIR, exist_ok=True)

                ## Retrieving actual time to save the results file
                timenow  = datetime.datetime.now().strftime('%m_%d_%Y_%H_%M_%S')
                
                ## TRAIN RESULTS
                acc_train = pd.DataFrame(acc_train, index = ['MF', 'GRC'])
                acc_train.to_csv(RESULTS_DIR + '/train_' + str(timenow) + '.csv')

                ## VALIDATION RESULTS
                acc_val = pd.DataFrame(acc_val, index = ['MF', 'GRC'])
                acc_val.to_csv(RESULTS_DIR + '/val_' + str(timenow) + '.csv')

                ## TEST RESULTS
                acc_test = pd.DataFrame(acc_test, index = ['MF', 'GRC'])
                acc_test.to_csv(RESULTS_DIR + '/test_' + str(timenow) + '.csv')

                ## Saving all the params information in a file
                params_file_name = RESULTS_DIR + '/params_file.csv' 

                row_train = [seed, perceptron_seed, sigma, round(fraction, 2), noise, mf_grc_w, mf_goc_w, grc_goc_w, goc_grc_w, interval, N_epochs,
                                    C, num_patterns, n_seeds, gamma, batch_size,
                                    'train', 'train_' + str(timenow) + '.csv']
                row_test = [seed, perceptron_seed, sigma, round(fraction, 2), noise, mf_grc_w, mf_goc_w, grc_goc_w, goc_grc_w, interval, N_epochs,
                                    C, num_patterns, n_seeds, gamma, batch_size,
                                    'test', 'test_' + str(timenow) + '.csv']
                row_val = [seed, perceptron_seed, sigma, round(fraction, 2), noise, mf_grc_w, mf_goc_w, grc_goc_w, goc_grc_w, interval, N_epochs,
                                    C, num_patterns, n_seeds, gamma, batch_size,
                                    'val', 'val_' + str(timenow) + '.csv']

                
                # If dataframe does not yet exist
                if not os.path.isfile(params_file_name):

                    params_df = pd.DataFrame([row_train, row_test, row_val], columns = columns)
                    params_df.to_csv(params_file_name, index = 0)

                # If dataframe already exists
                else:
                    params_df = pd.read_csv(params_file_name)
                    params_df = pd.concat([params_df, pd.DataFrame([row_train, row_test, row_val], columns = columns)])
                    params_df.to_csv(params_file_name, index = 0)
